In [7]:
import json
import random
from http import HTTPStatus
import dashscope
import time
import pandas as pd
import os
from dashscope import Generation

In [8]:
# 读取测试文件
def read_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f.readlines()[1:]:  # 跳过表头
            parts = line.strip().split("\t")
            target, text = parts[1], parts[2]
            data.append((target, text))
    return pd.DataFrame(data, columns=["TARGET", "TEXT"])

In [9]:
# 生成 prompt
def generate_prompt(row):
    return (f"以下是一条用户针对\"{row['TARGET']}\"的评论。请判断这条评论的立场是'反对'、'中立'还是'支持'。"
            f"评论：\"{row['TEXT']}\""
            f"你的回答只需写“反对”、“中立”或“支持”。")

In [10]:
# 调用 ChatGPT API 进行批量预测
def batch_predict(prompts):
    results = []
    for prompt in prompts:
        print(prompt)
        responses = Generation.call(model=Generation.Models.qwen_turbo,
                            prompt='今天天气好吗？')
        
        if responses.status_code == HTTPStatus.OK:
            print('Result is: %s' % responses.output)
        else:
            print('Failed request_id: %s, status_code: %s, code: %s, message:%s' %
                    (responses.request_id, responses.status_code, responses.code,
                    responses.message))

        results.append(responses.output.strip())

        print(f"results:{results}")
    return results


In [15]:
# 文件路径
test_file_path = 'weibo-test.txt'
output_file_path = 'weibo-test-predictions.txt'

# 加载数据
test_data = read_data(test_file_path)

In [16]:
test_data

TARGET                                               TEXT
0     春节放鞭炮  #不放鞭炮过春节#杭州市城管委发布消息，大年初一早晨，杭州主城区环卫工人共清扫除夕夜市民燃放...
1     春节放鞭炮  我个人认为，春节不放鞭炮，是民俗被各路不作为所绑架的结果！鞭炮产生的雾霾能和那一根根大烟囱相...
2     春节放鞭炮  2013，十面霾伏，而今春节又近，又会有许多许多的人放鞭炮烟花，那么大家可不可劝告周围的人少...
3     春节放鞭炮  #今年过节不放鞭炮#过春节放鞭炮，虽然可以增加过年气氛，但是每年的鞭炮伤人的事件仍旧不少。而...
4     春节放鞭炮  #倡议：人口密集区禁放大炮和高空烟花#又是一年春节到，噼里啪啦的鞭炮声已不绝于耳，年味渐浓。...
..      ...                                                ...
995  深圳禁摩限电                              乘坐非法摩电车安全没保障，请大家共同抵制！
996  深圳禁摩限电  交通工具而已，有需要才诞生，问题出在使用的人，怎样搞好教育提高全民素质才是迫切需要的事，教育...
997  深圳禁摩限电  # 深圳禁摩 限电# 针对电动车集聚从事非法运营情况，口岸大队在福强沙嘴整治电动车、关爱生命...
998  深圳禁摩限电                                    这些电单车拉客的应该抓、太危险
999  深圳禁摩限电                        以后我们就不要在网上购物啦！快递费要加价，老百姓受罪。

[1000 rows x 2 columns]

In [17]:
# 生成 prompts
test_data["PROMPT"] = test_data.apply(generate_prompt, axis=1)

In [18]:
# 调用 API 进行预测
test_data["PREDICTION"] = batch_predict(test_data["PROMPT"].tolist())

# 保存结果
test_data.to_csv(output_file_path, sep="\t", index=False, encoding='utf-8')

print(f"预测完成，结果保存至 {output_file_path}")

以下是一条用户针对"春节放鞭炮"的评论。请判断这条评论的立场是'反对'、'中立'还是'支持'。评论："#不放鞭炮过春节#杭州市城管委发布消息，大年初一早晨，杭州主城区环卫工人共清扫除夕夜市民燃放的烟花爆竹垃圾113吨，比去年同期减少22%。虽然数量有所减少，但113吨还是一个不小的数字。今天杭州雾霾不轻，其实不放鞭炮过春节也挺好嘛～@浙江日报美丽乡村"你的回答只需写“反对”、“中立”或“支持”。


NameError: name 'openai' is not defined